# Calculate tax revenue growth of regions in Finland

Data source: vero.fi

Link: https://www.vero.fi/tietoa-verohallinnosta/tilastot/avoin_dat/

In [2]:
import pandas as pd
import numpy as np

In [3]:
# Create the base with region column DataFrame
base_df = pd.read_csv('../data/Indices/population_density_index.csv')
base_df = base_df[['Region code', 'Region name (en)', 'Region name (fi)']].copy()
base_df.drop_duplicates(inplace=True)
base_df.reset_index(drop=True, inplace=True)

base_df

,Region code,Region name (en),Region name (fi)
0,MK01,Uusimaa,Uusimaa
1,MK02,Southwest Finland,Varsinais-Suomi
2,MK04,Satakunta,Satakunta
3,MK05,Kanta-Häme,Kanta-Häme
4,MK06,Pirkanmaa,Pirkanmaa
5,MK07,Päijät-Häme,Päijät-Häme
6,MK08,Kymenlaakso,Kymenlaakso
7,MK09,South Karelia,Etelä-Karjala
8,MK10,South Savo,Etelä-Savo
9,MK11,North Savo,Pohjois-Savo


In [4]:
# Load tax data
TAX_DF = pd.read_pickle('../data/tax_data/full_tax_data_with_region_and_industry.pkl.zip')
TAX_DF

,Year,Business_ID,Name,Tax_Region,Tax_Revenue,Total_Amount_Paid,Tax_Advance,Tax_Return,Residual_Tax,Municipality_Code,Municipality,Code of region,Name of region in Finnish,Section
0,2011,0568703-2,Brändö Lax Ab,035 BRÄNDÖ,857376.69,222917.94,222913.97,0.00,0.00,035,Ahvenanmaa,21.0,Ahvenanmaa,A
1,2011,0841712-6,Norrfjärden Ab,035 BRÄNDÖ,0.00,0.00,0.00,0.00,0.00,035,Ahvenanmaa,21.0,Ahvenanmaa,K
2,2011,0144666-1,Brändö Andelshandel,035 BRÄNDÖ,0.00,-15.66,5369.70,5385.36,0.00,035,Ahvenanmaa,21.0,Ahvenanmaa,G
3,2011,0144668-8,Jurmo Andelshandel,035 BRÄNDÖ,0.00,300.00,0.00,0.00,300.00,035,Ahvenanmaa,21.0,Ahvenanmaa,G
4,2011,0200418-7,Lappo Handelslag,035 BRÄNDÖ,0.00,0.00,0.00,0.00,0.00,035,Ahvenanmaa,21.0,Ahvenanmaa,G
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3622762,2021,2769288-7,Eran Oy,050 Eura,105356.40,21405.03,21405.03,0.00,0.00,050,Satakunta,4.0,Satakunta,M
3622763,2021,2751387-7,Taljan Hallinta Oy,091 Helsinki,2665.81,533.16,1110.86,577.70,0.00,091,Uusimaa,1.0,Uusimaa,L
3622764,2021,2766401-8,Trimmi Inspire Oy,837 Tampere,0.00,0.00,0.00,0.00,0.00,837,Pirkanmaa,6.0,Pirkanmaa,M
3622765,2021,2741109-6,Ompelimo Taito-Ommel Oy,091 Helsinki,3934.83,786.97,678.48,0.00,108.49,091,Uusimaa,1.0,Uusimaa,C


In [16]:
tax_df = TAX_DF[['Year', 'Tax_Revenue', 'Code of region']].copy()
tax_df.dropna(inplace=True)
tax_df['Code of region'] = tax_df['Code of region'].apply(lambda x: f'MK{int(x):02d}')
tax_df.columns = ['Year', 'Tax Revenue (EUR)', 'Region code']

# Choose rows with year from 2017 to 2021
tax_df = tax_df[tax_df['Year'] >= 2017].copy()
tax_df.sort_values(by=['Region code', 'Year'], inplace=True)
tax_df.reset_index(drop=True, inplace=True)
tax_df

,Year,Tax Revenue (EUR),Region code
0,2017,695719.00,MK01
1,2017,0.00,MK01
2,2017,0.00,MK01
3,2017,0.00,MK01
4,2017,0.00,MK01
...,...,...,...
1659077,2021,0.00,MK21
1659078,2021,0.00,MK21
1659079,2021,26528.32,MK21
1659080,2021,5000.00,MK21


In [17]:
# Sum the Tax Revenue for each region for each year into column 'Tax Revenue (EUR)'
tax_df = tax_df.groupby(['Region code', 'Year']).sum()
tax_df.reset_index(inplace=True)
tax_df

,Region code,Year,Tax Revenue (EUR)
0,MK01,2017,1.976216e+10
1,MK01,2018,1.962438e+10
2,MK01,2019,1.995284e+10
3,MK01,2020,1.820357e+10
4,MK01,2021,2.501388e+10
...,...,...,...
90,MK21,2017,1.635309e+08
91,MK21,2018,1.663758e+08
92,MK21,2019,1.763903e+08
93,MK21,2020,1.606967e+08


In [18]:
tax_df['Tax Revenue Absolute Growth (EUR)'] = tax_df.groupby('Region code')['Tax Revenue (EUR)'].diff()
tax_df['Tax Revenue Relative Growth (%)'] = tax_df['Tax Revenue Absolute Growth (EUR)'] / tax_df['Tax Revenue (EUR)'] * 100

tax_df

,Region code,Year,Tax Revenue (EUR),Tax Revenue Absolute Growth (EUR),Tax Revenue Relative Growth (%)
0,MK01,2017,1.976216e+10,NaN,NaN
1,MK01,2018,1.962438e+10,-1.377768e+08,-0.702070
2,MK01,2019,1.995284e+10,3.284587e+08,1.646175
3,MK01,2020,1.820357e+10,-1.749275e+09,-9.609519
4,MK01,2021,2.501388e+10,6.810317e+09,27.226150
...,...,...,...,...,...
90,MK21,2017,1.635309e+08,NaN,NaN
91,MK21,2018,1.663758e+08,2.844898e+06,1.709923
92,MK21,2019,1.763903e+08,1.001453e+07,5.677483
93,MK21,2020,1.606967e+08,-1.569355e+07,-9.765941


In [19]:
df = pd.merge(base_df, tax_df, on='Region code', how='inner')
df

,Region code,Region name (en),Region name (fi),Year,Tax Revenue (EUR),Tax Revenue Absolute Growth (EUR),Tax Revenue Relative Growth (%)
0,MK01,Uusimaa,Uusimaa,2017,1.976216e+10,NaN,NaN
1,MK01,Uusimaa,Uusimaa,2018,1.962438e+10,-1.377768e+08,-0.702070
2,MK01,Uusimaa,Uusimaa,2019,1.995284e+10,3.284587e+08,1.646175
3,MK01,Uusimaa,Uusimaa,2020,1.820357e+10,-1.749275e+09,-9.609519
4,MK01,Uusimaa,Uusimaa,2021,2.501388e+10,6.810317e+09,27.226150
...,...,...,...,...,...,...,...
90,MK21,Åland,Ahvenanmaa,2017,1.635309e+08,NaN,NaN
91,MK21,Åland,Ahvenanmaa,2018,1.663758e+08,2.844898e+06,1.709923
92,MK21,Åland,Ahvenanmaa,2019,1.763903e+08,1.001453e+07,5.677483
93,MK21,Åland,Ahvenanmaa,2020,1.606967e+08,-1.569355e+07,-9.765941


In [40]:
df.to_csv('../data/Indices/tax_revenue_index.csv', index=False)